<a href="https://colab.research.google.com/github/JoaquinGonzalezSimon/Data_science_and_ML_from_Medium/blob/main/221010_Hate_Speech_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This project was taken from the following link and I use it to practice my skills

https://thecleverprogrammer.com/2020/08/19/hate-speech-detection-model/

https://medium.com/coders-camp/180-data-science-and-machine-learning-projects-with-python-6191bc7b9db9

In [3]:
from google.colab import files
import io
import pandas as pd
import re
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
upload = files.upload()

Saving test.csv to test.csv


In [4]:
test = pd.read_csv(io.BytesIO(upload['test.csv']))

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


In [6]:
upload_train = files.upload()

Saving train.csv to train.csv


In [7]:
train = pd.read_csv(io.BytesIO(upload_train['train.csv']))
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [10]:
def clean_text(df, text_field):
  df[text_field] = df[text_field].str.lower()
  df[text_field] = df[text_field].apply(lambda i: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", i))
  return df

In [11]:
test_clean = clean_text(test, 'tweet')
test_clean.head()

,id,tweet
0,31963,studiolife aislife requires passion dedication...
1,31964,white supremacists want everyone to see the ...
2,31965,safe ways to heal your acne altwaystoheal h...
3,31966,is the hp and the cursed child book up for res...
4,31967,3rd bihday to my amazing hilarious nephew el...


In [12]:
train_clean = clean_text(train, 'tweet')
train_clean.head()

,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for lyft credit i cant use cause they...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation


In [13]:
train_majority = train_clean[train_clean.label == 0]
train_minority = train_clean[train_clean.label == 1]
train_minority_upsampled = resample(train_minority, replace=True, n_samples=len(train_majority))
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [15]:
pipeline_sgd = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()),
                         ('nb', SGDClassifier())
                         ])

In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(train_upsampled['tweet'], train_upsampled['label'])

In [17]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((44580,), (14860,), (44580,), (14860,))

In [20]:
model = pipeline_sgd.fit(xtrain, ytrain)
ypred = model.predict(xtest)

In [21]:
f1_score(ytest, ypred)

0.9645608628659476